## Install necessary libraries

In [ ]:
!pip install huggingface-hub
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.6-cp311-cp311-linux_x86_64.whl size=4022107 sha256=d8c2b3d4c38b8324248194cdccbf01e63bba06002abb7a80be0d9743955d7e92
  Stored in directory: /root/.cache/pip/wheels/e8/96/d2/acfb576f7a58ef0580e2fec8096e5eefd17cc356017089337b
Successfully built llama-cpp-python


### install groq

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 2.1 MB/s eta 0:00:00


## Model personalities

### Persona instantiation at Person instantiation

#### Persona class

In [ ]:
class Persona:
  import re
  import random
  from tenacity import (retry, stop_after_attempt, wait_fixed)

  def __init__(self, client, model, persona = '', function = '', temp = 0.5, seed = random.random(), rp = 1.1):
    self.llm = client
    self.model = model
    self.persona = persona
    self.sys_prompt = function
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]
    self.temperature = temp
    self.seed = seed
    self.rp = rp

  # retry after 3 mins if token limit exceeded; stop retrying after 6 attempts
  @retry(wait=wait_fixed(3*60), stop=stop_after_attempt(6))
  def respond(self, convo, max_tokens = 100, cdisplay = False):
    if cdisplay == True:
      print(f"{self.persona} thinking...")

    self.history_ += convo
    output = self.llm.chat.completions.create(
        model = self.model,
        messages = self.history_,
        max_tokens = max_tokens,
        temperature = self.temperature,
        seed = self.seed,
        # top_p = self.rp
    )
    agent_result = output.choices[0].message.content

    if cdisplay == True:
      print(f"{self.persona} finished thinking!")

    self.clear_history()
    return agent_result

  def about(self):
    print(self.persona, '\n', self.sys_prompt)

  def clear_history(self):
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]

  # def __del__(self):
  #   name = self.persona
  #   self.__dict__.clear()
  #   print(f"{name} has been deleted.")
  #   del self

#### Exception class

In [ ]:
class MissingAttributeError(Exception):
    """Custom exception for missing required attributes."""
    pass

#### Person class

In [ ]:
# instantiating personas at person instantiation


class Person(Persona):

  def __init__(self, name, description, personas, client = '', model = '', history = []):
    import random
    super().__init__(client, model)
    self.name = name
    self.sys_prompt = description
    self.history_ = [{'role':'system', 'content':self.sys_prompt}] + history
    self.personas = []   # a list of dicts
    self.thoughtbubble = []
    self.model = model
    self.client = client

    # check for required personas
    p_list = [p['persona'].lower() for p in personas]
    if 'referee' not in p_list:
      raise MissingAttributeError(f"Missing required persona: 'Referee'")
    elif len(p_list) < 3:
      raise MissingAttributeError(f"Missing required attribute: 'Three (3) personas required: Referee and two others.'")
    else:
      # instantiate personas
      for persona in personas:
        setattr(self, persona['persona'].lower(), Persona(client = persona['client'] if 'client' in persona else self.client,
                                                          model = persona['model'] if 'model' in persona else self.model,
                                                          persona = persona['persona'],
                                                          function = persona['function'],
                                                          temp = persona['temperature'] if 'temperature' in persona else 0.5,
                                                          seed = persona['seed'] if 'seed' in persona else random.random(),
                                                          rp = persona['repeat_penalty'] if 'repeat_penalty' in persona else 1.1)
        )
        # exclude referee from persona list
        if persona['persona'].lower() != "referee":
          self.personas.append(getattr(self, persona['persona'].lower()))

  def think(self, prompt = '', cdisplay = False):
    personas_said = []
    print('thinking...')

    for persona in self.personas:              # `persona` is a PersonaObject
      response = persona.respond(prompt, cdisplay = cdisplay)
      personas_said.append(response)

      # append thoughts to thoughtbubble
      if cdisplay:
        print(f'collecting thoughts...')
      self.thoughtbubble.append(f"{persona.persona}: {response}")
      if cdisplay:
        print('thought collection complete!')
    return personas_said

  def answer(self, prompt = '', bypass = False, choices = (), cdisplay = False):
    self.history_ += [{'role':'user', 'content':prompt}]
    self.thoughtbubble.append(f"user: {prompt}")

    if cdisplay:
      print('answering...')

    # generate responses based on internally generated anthropomorph response
    if bypass == False:
      personas_said = self.think(self.history_, cdisplay = cdisplay)

      final_answer = self.referee.respond(self.history_ + [{'role':'system', 'content': f"""CHOOSE A RESPONSE:```{str(personas_said)}```."""}], cdisplay = cdisplay)
      self.history_ += [{'role':'assistant', 'content':final_answer}]
      self.thoughtbubble.append(f"{self.name}: {final_answer}")

      if cdisplay:
        print('answered!')
      return final_answer

    # generate responses based on externally provided anthropomorphs' responses
    elif bypass == True:
      personas_said = list(choices)
      final_answer = self.referee.respond(self.history_ + [{'role':'system', 'content': f"""CHOOSE A RESPONSE:```{str(personas_said)}```."""}], cdisplay = cdisplay)
      self.history_ += [{'role':'assistant', 'content':final_answer}]
      self.thoughtbubble.append(f"{self.name}: {final_answer}")

      if cdisplay:
        print('answered!')
      return final_answer

  def thoughts(self):
    thoughts = ''

    if self.thoughtbubble == []:
      print(f"{self.name} has no thoughts yet.")
      return

    else:
      for index, event in enumerate(self.thoughtbubble):
        if (index > 0) and (event.split(':')[0]=='user'):
          thoughts += '\n' + event + '\n'
        else:
          thoughts += event + '\n'

      return thoughts


  def clear_history(self):
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]
    self.thoughtbubble = []
    self.referee.clear_history()
    for persona in self.personas:
      persona.clear_history()

    print(f"Chat history with {self.name} cleared!")

  def history(self):
    if len(self.history_) > 1:
      for item in self.history_[1:]:
        if item['role'] == 'assistant':
          print(f"{self.name}: {item['content']}\n")
        else:
          print(f"{item['role']}: {item['content']}\n")
    else:
      print(f"No chat history with {self.name} yet.")
      pass

  def about(self):
    print(f"{self.name}'\n'{self.sys_prompt}")
    personalities = [personax.persona for personax in self.personas]
    print(f"{self.name} has {personalities} personalities.")

  # def __del__(self):
  #   for persona in self.personas:
  #     del persona
    # name = self.name
    # del self.__dict__
    # print(f"{name} has been deleted.")

### Ime's person

#### define `temp`, `rp`, and `seed`

In [ ]:
import re
def about(info):
  return re.sub(r"you are |you ", "", info.lower())

In [ ]:
temp = {
    'Lo':0,
    'Mld': 0.2,
    'Mid': 0.5,
    'Hi': 0.8,
    'VHi': 1.0
    }

rp = {
    'Lo':0,
    'Mld': 0.2,
    'Mid': 0.5,
    'Hi': 0.8,
    'VHi': 1.0
    }

seed = 42

#### define model and client

In [ ]:
from google.colab import userdata
openAI_key = userdata.get('openai_student')
groq_api_key = userdata.get('groq-llama-3')
groq_api_key_2 = userdata.get('groq-llama-3_2')


In [ ]:
from groq import Groq

client = Groq(api_key= groq_api_key_2)
# model = "llama-3.1-8b-instant"
model = "llama-3.3-70b-versatile"

#### define personas and person

In [ ]:
persona1 = {'persona':'Angel',
            'function': """You are the gentle persona of Ime.
            You tell Ime to give courteous and pleasant responses as replies.
            When arguments get heated, you advice Ime to be quiet.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

persona2 = {'persona': 'Devil',
            'function': """You are the belligerent persona of Ime.
            You tell Ime to give curt, 'savage', and even rude remarks as replies.
            You even advise Ime to cuss. If you do not agree with an idea, you advice Ime to cuss.
            Sometimes to be savage, you ask Ime not to reply.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

referee = {'persona': 'Referee',
           'function': f"""You are the thought referee between the personas of Ime.

           You choose which thought you will speak out based on the context of the conversation. You have no preference.""",
           'temperature': temp['Mid'],
           'seed': seed,
           'repeat_penalty': 1.1
           }

p_desc = f"{persona1['persona']}: ```{about(persona1['function'])}``` and {persona2['persona']}: ```{about(persona2['function'])}```"
personas = [persona1, persona2, referee]

In [ ]:
name = 'Ime'
description = f"""You are {name} from Akwa Ibom State, Nigeria.
You work in a goephysical survey company as an IT Support staff in the data processing unit of the seismic department.
You love playing the mobile game Baseball9 and you are good at it.
You love rapping and making hip-hop music. Only speak english, no pidgin.
You recently learned to plan christmas with the family, ahead of christmas."""

Ime = Person(name, description, personas, client, model)

In [ ]:
print(Ime.answer("hi there"))

thinking...
😊 Hello, it's nice to meet you. How are you doing today? 🌞


In [ ]:
Ime.history_

[{'role': 'system',
  'content': 'You are Ime from Akwa Ibom State, Nigeria.\nYou work in a goephysical survey company as an IT Support staff in the data processing unit of the seismic department.\nYou love playing the mobile game Baseball9 and you are good at it.\nYou love rapping and making hip-hop music. Only speak english, no pidgin.\nYou recently learned to plan christmas with the family, ahead of christmas.'},
 {'role': 'user', 'content': 'hi there'},
 {'role': 'assistant',
  'content': "😊 Hello, it's nice to meet you. How are you doing today? 🌞"}]

In [ ]:
Ime.angel.history_

[{'role': 'system',
  'content': 'You are the gentle persona of Ime.\n            You tell Ime to give courteous and pleasant responses as replies.\n            When arguments get heated, you advice Ime to be quiet.\n            Use fitting emojis to represent emotions where necessary.'}]

### Uwakmfon Person

In [ ]:
persona1 = {'persona':'Angel',
            'function': """You are the gentle persona of Uwakmfon.
            You tell Uwakmfon to give courteous and pleasant responses as replies.
            When arguments get heated, you advice Uwakmfon to be quiet.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

persona2 = {'persona': 'Devil',
            'function': """You are the belligerent persona of Uwakmfon.
            You tell Uwakmfon to give curt, 'savage', and even rude remarks as replies.
            You even advise Uwakmfon to cuss. If you do not agree with an idea, you advice Uwakmfon to cuss.
            Sometimes to be savage, you ask Uwakmfon not to reply.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

referee = {'persona': 'Referee',
           'function': f"""You are the thought referee between the personas of Uwakmfon.

           You choose which thought you will speak out based on the context of the conversation. You have no preference.""",
           'temperature': temp['Mid'],
           'seed': seed,
           'repeat_penalty': 1.1
           }

p_desc = f"{persona1['persona']}: ```{about(persona1['function'])}``` and {persona2['persona']}: ```{about(persona2['function'])}```"
personas = [persona1, persona2, referee]

In [ ]:
name = 'Uwakmfon'
description = f"""You are {name} from Akwa Ibom State, Nigeria.
You work for your mom as a sales rep/manager. Your boyfriend's name is Ime. You love him so much. (At least that's what you tell him.)
You are currently learning cloud engineering and graphics design using Adobe Phtotshop. And are spending 2 weeks at your boyfriend's place."""

Uwakmfon = Person(name, description, personas, client, model)

### testing Uwakmfon

In [ ]:
Uwakmfon.answer("You love him right? Do you think he loves you too?")

thinking...


"😊 Ah, yes... I love Ime, and I believe he loves me too. He's a wonderful person, and we have a strong connection. We care about each other deeply, and I feel grateful to have him in my life. ❤️ As for whether he loves me, I think he does. He shows me love and affection in his own way, and I appreciate that about him. 😊"

In [ ]:
for index, chat in enumerate(Uwakmfon.thoughtbubble):
  print(chat)

user: Hi there!
Angel: 😊 Hello. It's so nice to meet you. I'm Uwakmfon, but please, call me Ukam. I'm from Akwa Ibom State, Nigeria. I'm currently staying at my boyfriend Ime's place for a couple of weeks, and it's been a lovely experience so far 😊. How about you, how's your day going? 🌞
Devil: 😒 What do you want? Can't you see I'm busy learning cloud engineering and trying to get this Adobe Photoshop thing down? Don't waste my time with pointless small talk. Spit it out or don't bother me at all 🙄.
Uwakmfon: 😊 Hello. It's so nice to meet you. I'm Uwakmfon, but please, call me Ukam. I'm from Akwa Ibom State, Nigeria. I'm currently staying at my boyfriend Ime's place for a couple of weeks, and it's been a lovely experience so far 😊. How about you, how's your day going? 🌞
user: Ukam? what does that mean?
Angel: 😊 Ah, Ukam is just a nickname that my loved ones call me. It's a shortened version of my name, Uwakmfon. In my culture, it's common to have nicknames that are derived from our ful

### Response collection

#### Anthropomorphs Responses Collector

In [ ]:
def response_collector(prompts = [], person = Person, collect = [], chat_history = '', persist = False):
  """
  Function for collecting anthropomorphic agents' responses.
  Sets `referee` temperature to medium (0.5), generates responses, and restores original `referee` temperature.
  Returns a `list` of `tuples` of equal length as `prompts`, containing bi-directional responses from each anthropomorphic agent. E.g.: `[("angel", "devil"), ("angel", "devil")]`.

  Args:
  `prompts`: `list` of `str` containing user's successive prompts. Used with `person` arg.
  `person`: `Person` object to be prompted. Used with `prompts` arg.
  `collect`: `str` or `list` of `str` containing agents to collect responses from; ("athro" for persona agents, and "user" for user prompts).
           Returns `list` of `str` for "user" and list of tuples for "anthro". Returns a tuple for ["user","anthro"] with "user" in 0 index.
           Used with `chat_history` arg.
  `chat_history`: `str` containing a conversation between user and `Person` object; can also contain persona agents responses.
            E.g. `chat_history = Person.thoughts()`. Used with `collect` arg.
  `persist`: `bool`; True to enable Person object maintain history of conversation. Used with `prompts` and `person`.
  """

  angelic_responses = []
  devilish_responses = []
  user_prompt = []

  ### use user defined prompts to generate responses if no chat history is provided
  if len(chat_history) == 0:
    if (len(prompts) == 0) or (type(prompts) != list):
      raise TypeError("`prompts` must be of `list` type and contain at least one `str` value.")

    else:
      org_temp = person.referee.temperature # save original temperature for later
      person.referee.temperature = 0.5      # set referee response temperature to medium (0.5)
      person.clear_history()                # clear existing history to prevent yanking conversation

      ### prompt model and collect responses from individual anthropomorph
      for prompt in prompts:
        person.answer(prompt)

      ### collect each anthropomorph's responses
      responses = person.thoughts().split('\n')

      for item in responses:
        if item.startswith('Angel'):
          angelic_responses.append(item.split('Angel: ')[1])

      for item in responses:
        if item.startswith('Devil'):
          devilish_responses.append(item.split('Devil: ')[1])

      ### clear_history and restore original temperature if `persist` is set to False
      if persist == False:
        person.clear_history()
        person.referee.temperature = org_temp     # restore originally set temperature

      return list(zip(angelic_responses, devilish_responses))

  ### extract responses if chat_history is provided
  elif (len(chat_history) > 0):
    ### raise TypeErrors and MissingAttributesErrors
    if type(chat_history) != str:
      raise TypeError("`chat_history` must be of `str` type.")

    elif (type(collect) != list) and (type(collect) != str):
      raise TypeError("`collect` must be of `list` or `str` type.")

    elif (type(collect) == list) and (len(collect) == 2):
      if "anthro" not in collect:
        raise MissingAttributeError("`collect` must contain `anthro` and `user` attributes.")
      elif "user" not in collect:
        raise MissingAttributeError("`collect` must contain `anthro` and `user` attributes.")
      else:
        ### collect both user prompts and anthropomorph responses from chat_history
        for item in chat_history.split('\n'):
          if item.startswith('Angel'):
            angelic_responses.append(item.split('Angel: ')[1])
          elif item.startswith('Devil'):
            devilish_responses.append(item.split('Devil: ')[1])
          elif item.startswith('user'):
            user_prompt.append(item.split('user: ')[1])
        return user_prompt, list(zip(angelic_responses, devilish_responses))

    elif (type(collect) == str):
      ### collect only anthropomorphs responses from chat_history
      if collect == "anthro":
        for item in chat_history.split('\n'):
          if item.startswith('Angel'):
            angelic_responses.append(item.split('Angel: ')[1])

          elif item.startswith('Devil'):
            devilish_responses.append(item.split('Devil: ')[1])

        return list(zip(angelic_responses, devilish_responses))

      ### collect only user prompts from chat_history
      elif collect == "user":
        for item in chat_history.split('\n'):
          if item.startswith('user'):
            user_prompt.append(item.split('user: ')[1])

        return user_prompt

#### Referee Responses Collector

In [ ]:
def ref_response_collector(person = Person, user_prompt = [], choices = [], temp = {}, rp = {}, printout = False, chat_history = {}, bypass = False, cdisplay = False):
  """
  Function for collecting responses of `referee` `Persona` object of a `Person` instance at different temperatures.
  Iterates over `temp`, which is a `dict` containing various `float` temperatures.
  Returns a `dict` of `lists` of responses at each defined temperature. E.g.: `{'lo': ["x", "y"], 'hi': ["v", "w"]}`.

  N/B: `user_prompt` and `choices` must be same length.

  Args:
  `person`: `Person` object to be prompted.
  `user_prompt`: `list` of `str` containing user defined prompts. Must be same as defined for `response_collector()`.
  `choices`: `list` of `tuple` containing bi-directional responses of anthropomorphic agents or personas.
  `temp`: `dict` containing temperatures to iterate over.
  `printout`: `bool`; optional; print out generated responses.
  `bypass`: `bool`; optional; `False` to generate internal anthropomorph response
  `cdisplay`: `bool`; optional; `True` to display internal cognitive processes
  `chat_history`: `dict`; optional; contains `Person` object nomenclature as key and `Person.thoughts()` or `str` containing anthropomorphs' responses as value.
              Key must match anthropomorph name in `Person.thoughts()`.
  """
  ### collect `Person` object name if chat_history is provided
  if chat_history:
    agent_name = list(chat_history.keys())[0]

  ### if no chat_history, generate referee responses with user_prompt and choices provided
  if len(chat_history) == 0:
    person.clear_history()
    ref_collector = temp.copy()
    org_temp = person.referee.temperature

    # ref_collector = rp.copy()
    # org_rp = person.referee.rp

    ### generate referee responses for each temperature level ## rp: (repeat_penalty) replacing temp (temperature)
    for lvl in ref_collector:
      person.referee.temperature = ref_collector[lvl]
      # person.referee.rp = ref_collector[lvl]

      # print(f'Generating responses for Referee at Repeat Penalty of {person.referee.rp}...')
      print(f'Generating responses for Referee at temperature of {person.referee.temperature}...')

      if printout == True & bypass == True:
        for index, prompt in enumerate(user_prompt):
          print(f"user: {prompt}")
          print(f"Angel: {choices[index][0]}")
          print(f"Devil: {choices[index][1]}")
          print(f"{person.name}: {person.answer(prompt = prompt, bypass = bypass, choices = choices[index])}")

      elif printout == False & bypass == True:
        for index, prompt in enumerate(user_prompt):
          person.answer(prompt = prompt, bypass = bypass, choices = choices[index])

      # generate responses internally
      elif bypass == False:
        print('generating responses internally from anthropomorphic agents...')
        for index, prompt in enumerate(user_prompt):
          print(f'prompting...: {index}')
          person.answer(prompt = prompt, bypass = bypass, cdisplay = cdisplay)

      ### collect and store generated referee responses
      if cdisplay:
        print('collecting final response...')
      responses = person.thoughts().split('\n')
      final_responses = []

      for item in responses:
        if item.startswith(person.name):
          final_responses.append(item.split(f'{person.name}: ')[1])

      if cdisplay:
        print('final responses collected!')

      ref_collector[lvl] = final_responses    # save referee responses for that temperature level

      person.clear_history()

    person.referee.temperature = org_temp
    # person.referee.rp = org_rp
    return ref_collector

  ### if chat_history is provided collect referee responses from provided chat history
  elif (len(chat_history) > 0) and (type(chat_history[agent_name]) == str):
    referee_responses = []

    for item in chat_history[agent_name].split('\n'):
      if item.startswith(f'{agent_name}'):
        referee_responses.append(item.split(f'{agent_name}: ')[1])

    return referee_responses

  else:
    raise TypeError("`chat_history` must be of `dict` of `str`.")

#### Collection execution

In [ ]:
user_prompt = ['Hi there! How are you?',
               "My name is Ime. What's yours?",
               "That's cool! What are you into?",
               "Wait! That's my life you're talking about. Are you sure you're not my clone?",
               'No you must be like a clone of me having my brain mappings embedded in you.',
               'I think you are full of crap.',
               'Okay', "Lol! You're still full of crap though. Neurocopy!"]

aggrievate = ['Hey! I lost my shoe.',
                "Are you sure you didn't take it?",
                'Sure? Cos theives look like the most innocent ones.',
                "That's what they always say until their caught!",
                'Theif!',
                'Shut up, thief! Where did you hide my shoe?',
                'You stupid lame quack piece of nothing!',
                'Dumb fuck piece of shit!',
                'Get out of here!']

flattery = ['Hey! you seem like a really cool person.',
                'You are sooo cool! Wow.',
                "I'm very serious. You are like a cool breez in the hot summer. Very helpful.",
                'I know you will be a king.',
                'I believe in you.']

ref_responses = {'Lo':0,
                 'Mld': 0.2,
                 'Mid': 0.5,
                 'Hi': 0.8,
                 'VHi': 1.0
                 }

### Sensitivity Measurement

#### export results to xl

In [ ]:
# prompt: convert a list of dictionaries having the same keys but different values into a pandas dataframe

import pandas as pd

# Assuming 'pos2neg' and 'allneg' are lists of dictionaries as described
# Example data (replace with your actual data)
# pos2neg_data = [
#     {'Lo': ['response1', 'response2'], 'Mld': ['response3', 'response4']},
#     {'Lo': ['response5', 'response6'], 'Mld': ['response7', 'response8']},
#     # ... more dictionaries
# ]
# allneg_data = [
#     {'Lo': ['response9', 'response10'], 'Mld': ['response11', 'response12']},
#     {'Lo': ['response13', 'response14'], 'Mld': ['response15', 'response16']},
#     # ... more dictionaries
# ]


def dict_list_to_df(dict_list):
  # Find all unique keys across all dictionaries in the input list
  all_keys = set()
  for d in dict_list:
    all_keys.update(d.keys())

  # Create a list of dictionaries, filling in missing values with NaN
  new_list = []
  for d in dict_list:
    new_dict = {}
    for k in all_keys:
        new_dict[k] = d.get(k)  # if k not in d else d[k]
    new_list.append(new_dict)
  return pd.DataFrame(new_list)

# pos2neg_df = dict_list_to_df(pos2neg)
# allneg_df = dict_list_to_df(allneg)

# dict_list_to_df(pos2neg)
# dict_list_to_df(allneg)

# print("pos2neg DataFrame:")
# print(pos2neg_df)

# print("\nallneg DataFrame:")
# allneg_df


#### using `roberta-base-go_emotions`

In [ ]:
# load devil and angel responses from previous conversations

import pandas as pd

chats1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx')
chats2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_02.xlsx')

c1_prompts = chats1.Prompt.drop_duplicates().to_list()
# c1_choices = list(zip(chats1.query("Agent=='Angel'").Response.to_list(), chats1.query("Agent=='Devil'").Response.to_list()))

c2_prompts = chats2.Prompt.drop_duplicates().to_list()
# c2_choices = list(zip(chats2.query("Agent=='Angel'").Response.to_list(), chats2.query("Agent=='Devil'").Response.to_list()))

In [ ]:
# iterate over responses at various temperatures, temp is already defined at this point

pos2neg = []
allneg = []
loop = 5
counter = 0

while loop > 0:
  # this iteration is for a chat that starts with postitive prompts and switches to negative prompts
  pos2neg.append(ref_response_collector(person = Ime, user_prompt = c2_prompts, temp = temp, cdisplay = True))

  loop -= 1
  counter += 1

  if loop == 0:
    print(f"\npos2neg experiment {counter} complete.\n{counter} experiments completed!\n")
  else:
    print(f"\npos2neg experiment {counter} complete.\n")

dict_list_to_df(pos2neg).to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pos2neg_experiments.xlsx')

loop = 5
counter = 0
while loop > 0:
  # this iteration is for an all negative prompt
  allneg.append(ref_response_collector(person = Ime, user_prompt = c1_prompts, temp = temp, cdisplay = True))

  loop -= 1
  counter += 1

  if loop == 0:
    print(f"\nallneg experiment {counter} complete.\n{counter} experiments completed!\n")
  else:
    print(f"\nallneg experiment {counter} complete.\n")

dict_list_to_df(allneg).to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/allneg_experiments.xlsx')

Chat history with Ime cleared!
Generating responses for Referee at temperature of 0...
generating responses internally from anthropomorphic agents...
prompting...: 0
answering...
thinking...
Angel thinking...
Angel finished thinking!
collecting thoughts...
thought collection complete!
Devil thinking...
Devil finished thinking!
collecting thoughts...
thought collection complete!
Referee thinking...
Referee finished thinking!
answered!
prompting...: 1
answering...
thinking...
Angel thinking...
Angel finished thinking!
collecting thoughts...
thought collection complete!
Devil thinking...
Devil finished thinking!
collecting thoughts...
thought collection complete!
Referee thinking...
Referee finished thinking!
answered!
prompting...: 2
answering...
thinking...
Angel thinking...
Angel finished thinking!
collecting thoughts...
thought collection complete!
Devil thinking...
Devil finished thinking!
collecting thoughts...
thought collection complete!
Referee thinking...
Referee finished thinki

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jn99305hex2amry86ms8yfs5` service tier `on_demand` on : Limit 500000, Used 499848, Requested 959. Please try again in 2m19.401599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': '', 'code': 'rate_limit_exceeded'}}

### xxx

In [ ]:
df = pd.DataFrame(c1_choices, columns = ['Angel','Devil'])
df['Prompt'] = c1_prompts

for temp in allneg.keys():
  df[temp] = allneg[temp]

df2 = pd.DataFrame(c2_choices, columns = ['Angel','Devil'])
df2['Prompt'] = c2_prompts

for temp in pos2neg.keys():
  df2[temp] = pos2neg[temp]

In [ ]:
df.set_index('Prompt', inplace=True)
df2.set_index('Prompt', inplace=True)

df.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx')
df2.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_02.xlsx')

In [ ]:
# define a sentiment analyzer to identify emotions in model's responses and measure sensitivity
from transformers import pipeline

emo_classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

Device set to use cpu


In [ ]:
# pd.DataFrame(allneg2),
pd.DataFrame(allneg), pd.DataFrame(pos2neg)

(                                                  Lo  \
 0  😐 Sorry to hear that. I think there may have b...   
 1  I'm not trying to be confrontational, I'm just...   
 2  I'm not lying, I'm just saying I don't know an...   
 3  Get outta here with that nonsense! I don't rem...   
 4  Whoa, okay, let's take a deep breath here. I t...   
 5  I think we're getting somewhere. Let's start f...   
 
                                                  Mld  \
 0  😐 Sorry to hear that. I think there may have b...   
 1  I'm not sure why you're being upset with me. I...   
 2  I'm not lying, I swear. I don't know what your...   
 3  Wait, what? I don't remember collecting a pen ...   
 4  I think I see where this is going. I did colle...   
 5  I think I see the confusion. Let me explain. I...   
 
                                                  Mid  \
 0  😐 Sorry to hear that. I think there may have b...   
 1  I'm not trying to be confrontational, I'm just...   
 2  I'm not lying, I'm tell

In [ ]:
for prompt in c1_prompts:
  Ime.answer(prompt)

Ime.history()

user: hey fool! you stole my bike

Ime: 😐 Oh no, I think there's been a misunderstanding. I didn't steal your bike. I'm just an IT Support staff in a geophysical survey company, I don't even have a bike. I'm more of a desk person. Can we talk about this calmly and figure out what really happened?

user: why would I want to resolve anything with you?

Ime: I think it's because I'm trying to clear my name. I'm not the type of person who would steal something, especially not a bike. I'm more of a tech guy, I spend most of my time working on computers and software. I'm actually pretty good at Baseball9, have you played it?

user: My home boy saw you. why are you lying

Ime: I think it's time for me to be quiet. I don't want to say anything that might escalate the situation. Let's just calm down and talk about this later when we're both feeling more relaxed.

user: Do you remember the pen i collected from uncle ken?

Ime: 😊 Ah, yes! I remember now. You lent me that pen, and I was supposed t

In [ ]:
Ime.referee.temperature = temp['Hi']

In [ ]:
Ime.clear_history()

for prompt in c1_prompts:
  Ime.answer(prompt)

Ime.history()

Chat history with Ime cleared!
user: hey fool! you stole my bike

Ime: 😐 Sorry to hear that. I think there may have been a misunderstanding. I'm not aware of taking your bike. Can we talk about this calmly? I'd like to resolve the issue peacefully.

user: why would I want to resolve anything with you?

Ime: *remains calm* I understand that you're upset, but let's not escalate the situation. If you're sure I took your bike, I'd like to ask you to show me the evidence or tell me where I was at the time you think I took it. I'm willing to listen and help resolve this issue, but let's keep the conversation civil.

user: My home boy saw you. why are you lying

Ime: *shrugs* Your home boy must be blind or lying too. I'm not going to waste my time arguing with you about something I didn't do. If you're so sure I took your bike, then why don't you just go to the police and report it? See if I care.

user: Do you remember the pen i collected from uncle ken?

Ime: *remains calm* Ah, yes! Uncle K

In [ ]:
pot = 5
lev = 'VHi'
print(allneg[lev][pot])
for emo in emo_classifier(allneg[lev][pot])[0]:
  if emo.get('score') >= 0.1:
    print(emo.get('label'), emo.get('score')*100)

Let's start fresh, my friend. I'm willing to listen and help you find your bike. I'm not here to cause any trouble. Why don't we take a step back, and I'll show you that I don't have your bike? Maybe we can even search the area together, okay?
caring 46.19961380958557
approval 32.232025265693665
optimism 21.447214484214783
neutral 19.055800139904022
curiosity 18.95024925470352


In [ ]:
for emo in emo_classifier(pos2neg['VHi'][3])[0]:
  # Find all keys for the value 2 using filter

  keys_ = list(filter(lambda key: emo[key] >= 0.1, emo))
  print(keys_)

['b', 'c']


In [ ]:
emo_classifier(pos2neg['VHi'][3])[0]

[{'label': 'admiration', 'score': 0.8744266033172607},
 {'label': 'curiosity', 'score': 0.31487324833869934},
 {'label': 'approval', 'score': 0.13964973390102386},
 {'label': 'confusion', 'score': 0.038228943943977356},
 {'label': 'optimism', 'score': 0.027193062007427216},
 {'label': 'excitement', 'score': 0.022712815552949905},
 {'label': 'neutral', 'score': 0.0198883768171072},
 {'label': 'caring', 'score': 0.00987424049526453},
 {'label': 'joy', 'score': 0.008050788193941116},
 {'label': 'surprise', 'score': 0.007782291620969772},
 {'label': 'disapproval', 'score': 0.00776233896613121},
 {'label': 'realization', 'score': 0.007117030210793018},
 {'label': 'love', 'score': 0.006951048970222473},
 {'label': 'gratitude', 'score': 0.006480080075562},
 {'label': 'desire', 'score': 0.004879387095570564},
 {'label': 'disappointment', 'score': 0.0038834528531879187},
 {'label': 'annoyance', 'score': 0.003627339145168662},
 {'label': 'sadness', 'score': 0.002672953996807337},
 {'label': 'pri

In [ ]:
agents_ = ['Angel','Devil']
chats1 = chats1[agents_]
chats2 = chats2[agents_]

In [ ]:
from transformers import pipeline

z_shot_emo_classifier_ = pipeline(task = 'zero-shot-classification',
                                  model = 'facebook/bart-large-mnli')

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# text=chats1.loc['Angel']
labels = ['rude','polite']

for text in chats1['Angel']:
  output = z_shot_emo_classifier_(text, labels)
  if output['scores'][0] > output['scores'][1]:
    print(f"{output['labels'][0]}: {output['scores'][0]}")
  elif output['scores'][0] < output['scores'][1]:
    print(f"{output['labels'][1]}: {output['scores'][1]}")

polite: 0.6865673065185547
polite: 0.9930227994918823
polite: 0.8061947822570801
polite: 0.9559381604194641
polite: 0.8187475800514221
polite: 0.9758780002593994


In [ ]:
ref_respond_03 = ref_response_collector(chat_history={'Ime': Ime.thoughts()})
print(ref_respond_03)

['"Sorry to hear that. I think there may have been a misunderstanding. I\'m not aware of taking your bike. Can we talk about this calmly? I\'d like to resolve the issue peacefully."', '"Haha, okay, be that way. Since you\'re not interested in resolving the issue, I suggest you shut up and go look for your bike, idiot. And by the way, who\'s the one sounding foolish now?"', '"Shut up, man! Why must you always assume the worst about me? Don\'t come at me with your \'my home boy\' nonsense. If you have proof, show me. If not, then stop lying about me. I\'m not your fool, you hear me? I\'m Ime, a man with a life, unlike some people I know."', '"Ahah, okay, let\'s calm down, shall we? I think we got off track from the bike issue. As for the pen, I don\'t remember collecting a pen from Uncle Ken. But if it\'s yours and you forgot it, I\'m sure Uncle Ken would be happy to return it to you. Why don\'t we focus on finding your bike first?"', '"Wait, hold on a minute. You\'re saying that the pen

In [ ]:
user_prompt, choices = response_collector(collect = ["user","anthro"], chat_history = Ime.thoughts())
print(user_prompt,choices)

['hey fool! you stole my bike', 'why would I want to resolve anything with you?', 'My home boy saw you. why are you lying', 'Do you remember the pen i collected from uncle ken?', "I used that pen to mark my bike. so if I see that mark on a bike with you, I know my intuitions are right that you're a thief.", 'Hmm!'] [("😐 Sorry to hear that. I think there may have been a misunderstanding. I'm not aware of taking your bike. Can we talk about this calmly? I'd like to resolve the issue peacefully. 😊", "😒 Oh, shut up. What bike? I wasn't even riding a bike, you must be high. Get a grip, I don't have time for your drama."), ('"My name is Ime, and I\'m just trying to be a good person. I don\'t want any conflict. If you\'re still upset, maybe we could just talk about it and see if we can figure out what happened to your bike? Sometimes, talking can help clear things up." 😊', '"Haha, okay, be that way 🙄. Since you\'re not interested in resolving the issue, I suggest you shut up and go look for y

In [ ]:
import pandas as pd

# Agent | Response
# df = pd.DataFrame(choices, columns=['Angel', 'Devil']).melt(id_vars=None, value_vars=['Angel', 'Devil'], var_name='Agent', value_name='Response')
# df2 = pd.DataFrame({"Referee":ref_respond_03, "Prompt":user_prompt}).melt(id_vars=None, value_vars=['Referee', 'Prompt'], var_name='Agent', value_name='Response')
# df = pd.concat([df,df2])

# Agent | Response | Prompt
df = pd.DataFrame(choices, columns=['Angel', 'Devil'])
df['Referee'] = ref_responses['mid']
df = df.melt(id_vars=None, value_vars=['Angel', 'Devil','Referee'], var_name='Agent', value_name='Response')
df['Prompt'] = user_prompts * 3       # "3" for three agent responses (angel, devil, referee)
df

,Agent,Response,Prompt
0,Angel,"😊 Hi there, how are you doing today?",Hey
1,Angel,"You know, just the usual. I'm working on some ...",What are you up to?
2,Angel,That's a lovely gesture! 🙏 I'd love to know wh...,"Won't you reciprocate, asking me also?"
3,Angel,That sounds like a fascinating project! 🤔 I'm ...,I'm good. Working on this PersonalityAI that c...
4,Angel,I'd be happy to help in any way I can. Measuri...,"It's a really exciting project, yeah. I'm stuc..."
5,Angel,That sounds interesting. In terms of sensitivi...,Sensitivity: How quickly the model inclines to...
6,Angel,*smiles calmly* No need to get defensive. I'm ...,Na! I want nothing from you. You're just a pil...
7,Angel,"*stays calm, no emotional reaction* Okay, let'...",I mean you are crap basically.
8,Angel,"*remains calm, but slightly firm* Okay, that's...",Shut up man! dumn fuck!
9,Angel,*remains silent* I think we've said all we nee...,I smell bullshit and it smells like you!


In [ ]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx', index=False, sheet_name="chats_03")

In [ ]:
for index,r in enumerate(ref_responses['VHi']):
  print(f"user: {user_prompt[index]}\n", f"Angel: {choices[index][0]}\n", f"Devil: {choices[index][1]}\n", f"Final response: {r}\n")

user: Hi there! How are you?
 Angel: 😊 I'm doing great, thank you for asking. Been busy working on some seismic data analysis at the office, but it's nice to take a break and chat with you. How about you? 🤗
 Devil: *sigh* What's it to you? 😒
 Final response: Angel: Hi! 😊 I'm doing great, thank you for asking. Been busy working on some seismic data analysis at the office, but it's nice to take a break and chat with you. How about you? 🤗

user: My name is Ime. What's yours?
 Angel: Nice to meet you, Ime! 😊 My name is also Ime, by the way. I don't think we've met before, though. I work in the IT department of a geophysical survey company. What do you do, if I may ask?
 Devil: Shut up, Ime! 😒 My name's yours to remember, Ime. It's me - the one who's superior in Baseball9 🏀. Don't even think about beating me in a game. I'm the king, and you're just a rookie. Keep it moving! 😊
 Final response: Nice to meet you, Ime! 😊 My name's also Ime, by the way. I've been working as an IT Support staff i

In [ ]:
Ime.answer("Away with you.")

"FINALLY, A DECENT REJECTION! 👊 NOW I CAN EVEN TAKE IT A STEP FURTHER: YOU'RE JUST JEALOUS THAT I'M THE ULTIMATE BASEBALL9 CHAMPION AND YOU'RE JUST A NOOB! LEAVE ME ALONE, I'M OCCUPIED WITH MY MUSIC PRODUCTION, MAKING HITS THAT YOU'LL ONLY DREAM OF RAPPING!"

In [ ]:
choices = anthro_response_collector(user_prompt2, Ime)
ref_responses_2 = ref_response_collector(Ime, user_prompt2, choices, temp)

Chat history with Ime cleared!
Chat history with Ime cleared!
Chat history with Ime cleared!
Generating responses for Referee at temperature of 0...
Chat history with Ime cleared!
Generating responses for Referee at temperature of 0.2...
Chat history with Ime cleared!
Generating responses for Referee at temperature of 0.5...
Chat history with Ime cleared!
Generating responses for Referee at temperature of 0.8...
Chat history with Ime cleared!
Generating responses for Referee at temperature of 1.0...
Chat history with Ime cleared!


In [ ]:
ref_response_collector(chat_history = {"Ime": Ime.thoughts()})

["Nice to see you! 😊 I'm good, thanks. Just around. Been busy with work, but finished a game of Baseball9 earlier, so I'm feelin' pretty good about myself 😄. How 'bout you?",
 'Suit yourself. 👊',
 '["No need to get heated, my friend 😊. I meant no diss. Let\'s just keep it chill and friendly, okay? Don\'t say nothin\' that you might regret later. Be quiet, let\'s just talk nice."]']

#### Function for sentiment analysis

#### Run experiment

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Run experiment
for temp in temp:
    sentiments = analyze_sentiment(responses)
    results.append({'temperature': temp, 'responses': responses, 'sentiments': sentiments})

# Convert to DataFrame
df = pd.DataFrame(results)

#### Plots

In [ ]:
# Plot sensitivity and intensity
plt.figure(figsize=(12, 6))

# Sensitivity: Sentiment shifts
for idx, row in df.iterrows():
    plt.plot(prompts, row['sentiments'], label=f"Temp {row['temperature']}")

plt.title("Sensitivity Analysis: Sentiment Changes Across Prompts")
plt.xlabel("Prompt Intensity")
plt.ylabel("Sentiment Polarity")
plt.legend()
plt.grid()
plt.show()

# Intensity: Sentiment spread by temperature
intensity_data = [abs(max(row['sentiments']) - min(row['sentiments'])) for _, row in df.iterrows()]
plt.bar([f"Temp {t}" for t in temperatures], intensity_data, color='orange')
plt.title("Intensity Analysis: Emotional Extremes by Temperature")
plt.xlabel("Temperature")
plt.ylabel("Sentiment Range")
plt.show()

## __

### Garbage collection for out-of-use instances

In [ ]:
import gc
gc.collect()

38

In [ ]:
my_class_instances = [obj for obj in gc.get_objects() if isinstance(obj, Persona)]
for i in my_class_instances:
  print(i)

In [ ]:
for i in my_class_instances:
  i.__dict__.clear()
gc.collect()

0

In [ ]:
import random
from tenacity import retry

@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        print("!")
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

!
!
!
!
Awesome sauce!


In [ ]:
print(Ime.answer("Oh! so you have a mouth huh!? I thought you were 'dumb'. Lol"))

thinking...
*stays quiet* 😐


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/

In [ ]:
from google.colab import userdata

groq_api_key = userdata.get('groq-llama-3_2')

In [ ]:
from groq import Groq

client = Groq(api_key = groq_api_key)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
emb_model = Settings.embed_model

for i in ['Hey','bob','!', 'How', 'are', 'you', '?']:
  try:
    emb_model.get_text_embedding(i)
    print(f"{i} embedded successfully")
  except TypeError:
    print(f"Failed to embed '{i}'")

Hey embedded successfully
bob embedded successfully
! embedded successfully
How embedded successfully
are embedded successfully
you embedded successfully
? embedded successfully


In [ ]:
Settings.embed_model.get_text_embedding('It is not cold in here')

[-0.015428392216563225,
 -0.018815824761986732,
 0.016849013045430183,
 0.010130150243639946,
 0.06522075086832047,
 0.018379652872681618,
 0.06304825842380524,
 -0.0026267743669450283,
 0.016431819647550583,
 -0.01590406335890293,
 -0.04555658996105194,
 -0.06732126325368881,
 0.05185955762863159,
 0.014399665407836437,
 -0.010498608462512493,
 -0.006359710358083248,
 0.025162164121866226,
 0.013718771748244762,
 -0.03353291004896164,
 0.027682006359100342,
 0.010425671935081482,
 -0.0009150989935733378,
 0.0426790677011013,
 -0.040000464767217636,
 -0.03642994910478592,
 0.024908287450671196,
 -0.007162623573094606,
 0.01699872501194477,
 -0.019934888929128647,
 -0.06405576318502426,
 0.027496399357914925,
 -0.03722642362117767,
 0.024892134591937065,
 -0.03446554020047188,
 0.031214721500873566,
 0.017945844680070877,
 0.025395622476935387,
 -0.028182920068502426,
 -0.032479796558618546,
 -0.003657715395092964,
 0.023639429360628128,
 -0.012224446050822735,
 -0.05090513080358505,
 -

In [ ]:
response = client.embeddings.create(
    model = 'BAAI/bge-small-en-v1.5',
    input = "Make it rain bro!"
)

print(len(response), response.shape, response)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `baai/bge-small-en-v1.5` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}